In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2bb9aee4b9cc2cfbb6d345d79f97c0d8ce5cc1fbcdd1e995bf8910cdbd5adc24
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').appName('jean').getOrCreate()
# spark = SparkSession.builder.master('spark://127.0.0.1:7077').appName('jean').getOrCreate()
sc = spark.sparkContext

In [ ]:
sc.addFile('https://www.dropbox.com/s/yviaftku1t09ejx/bible.txt?dl=1')

In [ ]:
# carregando o arquivo
rdd = sc.textFile('file://' + SparkFiles.get('bible.txt'))

In [ ]:
rdd.take(5)

['1:1 In the beginning God created the heaven and the earth.',
 '',
 '1:2 And the earth was without form, and void; and darkness was upon',
 'the face of the deep. And the Spirit of God moved upon the face of the',
 'waters.']

In [ ]:
# fazendo split de sentenças em palavras
palavras = rdd.flatMap(lambda frase: frase.split(' '))

In [ ]:
type(palavras)

pyspark.rdd.PipelinedRDD

In [ ]:
palavras.take(10)

['1:1',
 'In',
 'the',
 'beginning',
 'God',
 'created',
 'the',
 'heaven',
 'and',
 'the']

In [ ]:
palavras.count()

851195

In [ ]:
palavras.countByValue()

### Aeroportos

In [ ]:
sc.addFile('https://www.dropbox.com/s/3k3v5tol6mypy35/airports.text?dl=1')

In [ ]:
# carregando o arquivo
rdd = sc.textFile('file://' + SparkFiles.get('airports.text'))

In [ ]:
rdd.take(5)

['1,"Goroka","Goroka","Papua New Guinea","GKA","AYGA",-6.081689,145.391881,5282,10,"U","Pacific/Port_Moresby"',
 '2,"Madang","Madang","Papua New Guinea","MAG","AYMD",-5.207083,145.7887,20,10,"U","Pacific/Port_Moresby"',
 '3,"Mount Hagen","Mount Hagen","Papua New Guinea","HGU","AYMH",-5.826789,144.295861,5388,10,"U","Pacific/Port_Moresby"',
 '4,"Nadzab","Nadzab","Papua New Guinea","LAE","AYNZ",-6.569828,146.726242,239,10,"U","Pacific/Port_Moresby"',
 '5,"Port Moresby Jacksons Intl","Port Moresby","Papua New Guinea","POM","AYPY",-9.443383,147.22005,146,10,"U","Pacific/Port_Moresby"']

In [ ]:
# filtrando linhas dos USA
rdd_usa = rdd.filter(lambda l: l.split(',')[3] == '\"United States\"')

In [ ]:
# rdd_usa.collect()

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [ ]:
# rdd_amostra = rdd.sample(False, 0.1)

In [ ]:
# rdd_amostra.collect()

In [ ]:
# calculando a media (reduce -> soma / count)
soma = rdd.reduce(lambda x, y: x + y)

In [ ]:
qtd = rdd.count()

In [ ]:
media = soma / qtd
media

5.5

In [ ]:
# Somando apenas os valores pares
soma = rdd.filter(lambda x: x % 2 == 0).reduce(lambda x, y: x + y)

In [ ]:
soma

30

In [ ]:
2+4+6+8+10

30

# Exercício 1

In [ ]:
rdd = sc.textFile('file://' + SparkFiles.get('airports.text'))

In [ ]:
# 1- filtrar aeroportos nos USA
rdd_usa = rdd.filter(lambda x: \
                     x.split(',')[3] == '"United States"')
# 2 - calcular a latitude media destes aeroportos
rdd_latitudes = rdd_usa.map(lambda x: float(x.split(',')[6]))

# somar todas as latitudes
soma_latitudes = rdd_latitudes.reduce(lambda x, y: x + y)
qtd_elementos = rdd_latitudes.count()

# media
media = soma_latitudes / qtd_elementos
media

41.44429158866408

# Exercicio 2

In [ ]:
rdd = sc.textFile('file://' + SparkFiles.get('bible.txt'))

In [ ]:
# 1) contar a frequencia de palavras que se iniciam com vogais
rdd_palavras = rdd.flatMap(lambda x: x.split(' '))

rdd_palavras_sem_vazio = rdd_palavras.filter(lambda x: len(x) > 0)
rdd_comeca_vogal = rdd_palavras_sem_vazio.filter(lambda x: x[0].lower() in 'aeiou')
rdd_comeca_vogal.count()

226451

In [ ]:
# 2) Quantas palavras se iniciam com cada vogal
rdd_comeca_vogal_1_caracter = rdd_comeca_vogal.map(lambda x: x.lower()[0])
rdd_comeca_vogal_1_caracter.countByValue()

defaultdict(int, {'a': 98347, 'e': 13178, 'i': 44286, 'o': 52501, 'u': 18139})

In [ ]:
# 3) contar a ocorrencia de cada vogal no texto todo

rdd_caracteres = rdd_palavras_sem_vazio.flatMap(lambda x: list(x))
rdd_caracteres_lc = rdd_caracteres.map(lambda x: x.lower())
rdd_caracteres_lc_vogais = rdd_caracteres_lc.filter(lambda x: x in 'aeiou')
rdd_caracteres_lc_vogais.countByValue()

defaultdict(int,
            {'a': 275701, 'e': 412160, 'i': 193926, 'o': 243143, 'u': 83462})

# Pair RDD

In [ ]:
rdd = sc.textFile('file://' + SparkFiles.get('airports.text'))

In [ ]:
#montando pair rdd no formato (cidade, pais)
prdd = rdd.map(lambda x: (x.split(',')[2], x.split(',')[3]))

In [ ]:
# remover aspas duplas de chave e valor
prdd_limpo = prdd.map(lambda x: (x[0].replace('"', ''), \
                                 x[1].replace('"', '')))

In [ ]:
# mantendo apenas elementos dos estados unidos
prdd_usa = prdd_limpo.filter(lambda x: x[1] == 'United States')

In [ ]:
prdd_usa.take(3)

[('Greencastle', 'United States'),
 ('Dowagiac', 'United States'),
 ('Cambridge', 'United States')]

In [ ]:
# convertendo paises para cx alta (MAP)
prdd_usa_uc = prdd_usa.map(lambda x: (x[0], x[1].upper()))

In [ ]:
prdd_usa_uc.take(3)

[('Greencastle', 'UNITED STATES'),
 ('Dowagiac', 'UNITED STATES'),
 ('Cambridge', 'UNITED STATES')]

In [ ]:
prdd_usa_uc = prdd_usa.mapValues(lambda x: x.upper())

In [ ]:
prdd_usa_uc.take(3)

[('Greencastle', 'UNITED STATES'),
 ('Dowagiac', 'UNITED STATES'),
 ('Cambridge', 'UNITED STATES')]

In [ ]:
prdd_usa_uc.coalesce(1).saveAsTextFile('prdd_.txt')

reducebykey

In [ ]:
rdd = sc.textFile('file://' + SparkFiles.get('bible.txt'))

In [ ]:
rdd.take(3)

['1:1 In the beginning God created the heaven and the earth.',
 '',
 '1:2 And the earth was without form, and void; and darkness was upon']

In [ ]:
palavras = rdd.flatMap(lambda x: x.split(' '))
palavras.countByValue()

In [ ]:
# gerando (palavra, 1)
prdd = palavras.map(lambda x: (x, 1))

In [ ]:
# reduzir (somar) as ocorrencias de cada palavra
prdd_ocorrencia = prdd.reduceByKey(lambda x, y: x + y)

In [ ]:
prdd_ocorrencia.take(10)

[('God', 2229),
 ('created', 38),
 ('', 27159),
 ('And', 12735),
 ('earth', 328),
 ('was', 4448),
 ('form,', 4),
 ('upon', 2718),
 ('of', 34547),
 ('deep.', 5)]

In [ ]:
prdd_ocorrencia_ordenado = prdd_ocorrencia.sortBy(lambda x: x[1], ascending=False)

In [ ]:
prdd_ocorrencia_ordenado.take(3)

[('the', 62252), ('and', 38641), ('of', 34547)]

# Solução: Preço médio por número de quartos

In [ ]:
sc.addFile('https://www.dropbox.com/s/5gy1beolwhn9isf/RealEstate.csv?dl=1')
rdd = sc.textFile('file://'+ SparkFiles.get('RealEstate.csv'))

In [ ]:
rdd.take(3)

['MLS,Location,Price,Bedrooms,Bathrooms,Size,Price SQ Ft,Status',
 '132842,Arroyo Grande,795000.00,3,3,2371,335.30,Short Sale',
 '134364,Paso Robles,399000.00,4,3,2818,141.59,Short Sale']

In [ ]:
rdd_limpo = rdd.filter(lambda x: not x.startswith('MLS'))

In [ ]:
prdd = rdd_limpo.map(lambda x: (int(x.split(',')[3]), \
                                (float(x.split(',')[2]), 1)))

In [ ]:
prdd.take(3)

[(3, (795000.0, 1)), (4, (399000.0, 1)), (4, (545000.0, 1))]

In [ ]:
somas = prdd.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [ ]:
somas.take(3)

[(4, (85575190.0, 177)), (2, (32761834.0, 123)), (0, (586900.0, 2))]

In [ ]:
medias = somas.mapValues(lambda x: x[0]/x[1])
medias = medias.sortByKey(ascending=True)

In [ ]:
medias.collect()

[(0, 293450.0),
 (1, 169981.81818181818),
 (2, 266356.3739837398),
 (3, 359062.20649651974),
 (4, 483475.6497175141),
 (5, 657858.0645161291),
 (6, 603225.0),
 (7, 325000.0),
 (10, 699000.0)]

### SparkSQL

In [ ]:
from pyspark.sql.functions import col, max, min, avg

In [ ]:
df = spark.read.csv('file://' + SparkFiles.get('airports.text'), 
                    inferSchema=True)

In [ ]:
df.show(5)

+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|_c0|                 _c1|         _c2|             _c3|_c4| _c5|      _c6|       _c7| _c8| _c9|_c10|                _c11|
+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|  1|              Goroka|      Goroka|Papua New Guinea|GKA|AYGA|-6.081689|145.391881|5282|10.0|   U|Pacific/Port_Moresby|
|  2|              Madang|      Madang|Papua New Guinea|MAG|AYMD|-5.207083|  145.7887|  20|10.0|   U|Pacific/Port_Moresby|
|  3|         Mount Hagen| Mount Hagen|Papua New Guinea|HGU|AYMH|-5.826789|144.295861|5388|10.0|   U|Pacific/Port_Moresby|
|  4|              Nadzab|      Nadzab|Papua New Guinea|LAE|AYNZ|-6.569828|146.726242| 239|10.0|   U|Pacific/Port_Moresby|
|  5|Port Moresby Jack...|Port Moresby|Papua New Guinea|POM|AYPY|-9.443383| 147.22005| 146|10.0|   U|Pacific/Port_Moresby|
+---+-----------

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



In [ ]:
df.createOrReplaceTempView('vw_aeroportos')

In [ ]:
query = 'SELECT _c2, _c3, _c6, _c7 FROM vw_aeroportos WHERE _c3 LIKE "United States"'
spark.sql(query).show(10)

+------------+-------------+----------+-----------+
|         _c2|          _c3|       _c6|        _c7|
+------------+-------------+----------+-----------+
| Greencastle|United States|39.6335556|-86.8138056|
|    Dowagiac|United States|41.9929342|-86.1280125|
|   Cambridge|United States|39.9750278|-81.5775833|
|Sturgeon Bay|United States|44.8436667|-87.4215556|
|Stewartstown|United States|39.7948244|-76.6471914|
|   Pendleton|United States|    45.695|-118.841389|
|      Tyonek|United States| 61.076667|-151.138056|
| Riverton WY|United States| 43.064167|-108.459722|
| Montrose CO|United States| 38.509794|-107.894242|
| Bolingbrook|United States|41.6959744|-88.1292306|
+------------+-------------+----------+-----------+
only showing top 10 rows



In [ ]:
# Quantidade de aeroportos por país
# SELECT COUNT(*) FROM vw_aeroportos GROUP BY country
df.groupBy('_c3').count().orderBy(col('count').desc()).show(10)

+--------------+-----+
|           _c3|count|
+--------------+-----+
| United States| 1697|
|        Canada|  435|
|       Germany|  321|
|     Australia|  263|
|        Russia|  249|
|        France|  233|
|         China|  219|
|        Brazil|  213|
|United Kingdom|  210|
|         India|  140|
+--------------+-----+
only showing top 10 rows



In [ ]:
# Latitude média dos aeroportos dos USA
# SELECT AVG(latitude) FROM view WHERE country = "United States"
df.filter('_c3 = "United States"').\
   select(avg(col('_c6'))).show(10)

+------------------+
|          avg(_c6)|
+------------------+
|41.444291588664015|
+------------------+



In [ ]:
df.filter(col('_c3').like('United States')).\
   select(avg(col('_c6'))).show(10)

+------------------+
|          avg(_c6)|
+------------------+
|41.444291588664015|
+------------------+



In [ ]:
df = df.withColumnRenamed('_c3', 'country')
df.show(5)

+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|_c0|                 _c1|         _c2|         country|_c4| _c5|      _c6|       _c7| _c8| _c9|_c10|                _c11|
+---+--------------------+------------+----------------+---+----+---------+----------+----+----+----+--------------------+
|  1|              Goroka|      Goroka|Papua New Guinea|GKA|AYGA|-6.081689|145.391881|5282|10.0|   U|Pacific/Port_Moresby|
|  2|              Madang|      Madang|Papua New Guinea|MAG|AYMD|-5.207083|  145.7887|  20|10.0|   U|Pacific/Port_Moresby|
|  3|         Mount Hagen| Mount Hagen|Papua New Guinea|HGU|AYMH|-5.826789|144.295861|5388|10.0|   U|Pacific/Port_Moresby|
|  4|              Nadzab|      Nadzab|Papua New Guinea|LAE|AYNZ|-6.569828|146.726242| 239|10.0|   U|Pacific/Port_Moresby|
|  5|Port Moresby Jack...|Port Moresby|Papua New Guinea|POM|AYPY|-9.443383| 147.22005| 146|10.0|   U|Pacific/Port_Moresby|
+---+-----------

# Exemplo com header

In [ ]:
sc.addFile('https://www.dropbox.com/s/cjztq44o5k7imwu/transactions_amostra.csv?dl=1')
df_tde = spark.read.csv('file://' + SparkFiles.get('transactions_amostra.csv'), \
                        sep=';', header=True, inferSchema=True)

In [ ]:
df_tde.printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- comm_code: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- flow: string (nullable = true)
 |-- trade_usd: long (nullable = true)
 |-- weight_kg: double (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
df_tde.show(5)

+---------------+----+---------+--------------------+---------+---------+---------+-------------------+--------+--------------------+
|country_or_area|year|comm_code|           commodity|     flow|trade_usd|weight_kg|      quantity_name|quantity|            category|
+---------------+----+---------+--------------------+---------+---------+---------+-------------------+--------+--------------------+
|        Belgium|2016|   920510|Brass-wind instru...|   Export|   571297|   3966.0|    Number of items|  4135.0|92_musical_instru...|
|      Guatemala|2008|   660200|Walking-sticks, s...|   Export|    35022|   5575.0|    Number of items| 10089.0|66_umbrellas_walk...|
|       Barbados|2006|   220210|Beverage waters, ...|Re-Export|    81058|  44458.0|   Volume in litres| 24113.0|22_beverages_spir...|
|        Tunisia|2016|   780411|Lead foil of a th...|   Import|     4658|    121.0|Weight in kilograms|   121.0|78_lead_and_artic...|
|      Lithuania|1996|   560110|Sanitary towels, ...|   Export